# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install scikit-learn
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments
from transformers import DataCollatorWithPadding
from peft import LoraConfig
from peft import get_peft_model
from peft import AutoPeftModelForSequenceClassification
from transformers import AutoModelForCausalLM
import numpy as np
from datasets import load_dataset
import torch
from sklearn.metrics import accuracy_score

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 68.2 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 36.9 MB/s eta 0:00:00


In [2]:
# https://huggingface.co/docs/transformers/training
# or load the separate splits if the dataset has train/validation/test splits
train_dataset = load_dataset("cmotions/NL_restaurant_reviews", split="train")
valid_dataset = load_dataset("cmotions/NL_restaurant_reviews", split="validation")
test_dataset  = load_dataset("cmotions/NL_restaurant_reviews", split="test")

#commented out code used for education and debug
#look at full dataset
#print("look at full dataset")
dataset = load_dataset("cmotions/NL_restaurant_reviews")
#print(dataset)
#print("\n")

#length of train split of dataset
#print("length of train split of dataset")
#print(len(train_dataset))
#print("\n")

#looks at characteristics of train
#print("Characteristics of train")
#print(train_dataset)
#print("\n")

# Inspect the first example. See what a restaurant review looks like
#print("Train first example")
#print(train_dataset[0])
#print("\n")

train_dataset
print("\n")
test_dataset

Generating train split:   0%|          | 0/116693 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/14587 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14587 [00:00<?, ? examples/s]

Dataset({
    features: ['restaurant_ID', 'restaurant_review_ID', 'michelin_label', 'score_total', 'score_food', 'score_service', 'score_decor', 'fame_reviewer', 'reviewscore_food', 'reviewscore_service', 'reviewscore_ambiance', 'reviewscore_waiting', 'reviewscore_value', 'reviewscore_noise', 'review_text', 'review_length'],
    num_rows: 14587
})

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

splits = ["train", "validation", "test"]

def preprocess_function(examples):
    tokenized = tokenizer(
        examples["review_text"],
        padding="max_length",
        truncation=True,
        max_length=base_model.config.max_position_embeddings,
    )
    tokenized["labels"] = examples["michelin_label"]
    return tokenized
base_model     = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "negative review", 1: "positive review"},
    label2id={"negative review": 0, "positive review": 1},
)
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        preprocess_function,   
        batched=True,
        remove_columns=dataset[split].column_names,
    )


# Freeze all the parameters of the base model
for param in base_model.base_model.parameters():
    param.requires_grad = False
    
print(base_model)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/116693 [00:00<?, ? examples/s]

Map:   0%|          | 0/14587 [00:00<?, ? examples/s]

Map:   0%|          | 0/14587 [00:00<?, ? examples/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [4]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}
    
    #precision = precision_score(labels, pred)
    #recall = recall_score(labels, pred)
    #f1 = f1_score(labels, pred)
    #return {"accuracy": acc}

# Training the base model
trainer_base = Trainer(
    model=base_model,
    args=TrainingArguments(
        output_dir="./data/base_model_training",
        learning_rate=2e-3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Training the base model
trainer_base.train()

# Evaluate the base model
base_model_evaluation = trainer_base.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.129900,0.118066,0.970316


Checkpoint destination directory ./data/base_model_training/checkpoint-7294 already exists and is non-empty.Saving will proceed but saved results may be invalid.


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [5]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_lin", "v_lin"], # found in printed model above
    task_type="SEQ_CLS",
    # installed version of PEFT, LoraConfig doesn’t accept adapter_name in its constructor (text copied from actual error) 
    # hence no adapter_name here, error is encountered if adapter is used here.
)
lora_model = get_peft_model(base_model, config)
lora_model.print_trainable_parameters()

trainable params: 1,331,716 || all params: 67,694,596 || trainable%: 1.967241225577297


In [6]:
lora_model.save_pretrained("distilbert-base-uncased-lora")
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("distilbert-base-uncased-lora")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Unfreeze all the model parameters for Fine-tuning 
for param in lora_model.parameters():
    param.requires_grad = True

training_args = TrainingArguments(
    output_dir="review_classifier",
    evaluation_strategy="epoch",
    num_train_epochs=1,            
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    push_to_hub=False,
)

In [9]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.073400,0.069464,0.978405


Checkpoint destination directory review_classifier/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=7294, training_loss=0.0904770862379563, metrics={'train_runtime': 5837.0609, 'train_samples_per_second': 19.992, 'train_steps_per_second': 1.25, 'total_flos': 1.5723145603313664e+16, 'train_loss': 0.0904770862379563, 'epoch': 1.0})

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [16]:
# Saving the model
lora_model.save_pretrained("/tmp/Mwesi_LightweightFineTuning")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [17]:
# Evaluate PEFT model
peft_model_evaluation = trainer.evaluate()

In [18]:
# Compare the results of the base model & PEFT model
print("\nPEFT Model Evaluation:")
print(peft_model_evaluation)
print("Base Model Evaluation:")
print(base_model_evaluation)


PEFT Model Evaluation:
{'eval_loss': 0.06946391612291336, 'eval_accuracy': 0.9784054294920135, 'eval_runtime': 246.1912, 'eval_samples_per_second': 59.251, 'eval_steps_per_second': 3.704, 'epoch': 1.0}
Base Model Evaluation:
{'eval_loss': 0.11806640028953552, 'eval_accuracy': 0.9703160348255295, 'eval_runtime': 239.6264, 'eval_samples_per_second': 60.874, 'eval_steps_per_second': 3.806, 'epoch': 1.0}
